In [1]:
%run modules.ipynb
# Change this cell to change the kernel function throughout
kernel_params = {"kernel": "rbf", "gamma": 1}
kernel_func = partial(rbf_kernel, gamma=1)
kernel_type = "gaussian"
import multiprocessing
from multiprocessing import Pool
from time import sleep

In [2]:
#dataset load
len_dataset=79855
load_profile('egazzarr')
qb = QueryBuilder()
qb.append(orm.StructureData)

In [3]:
select=1000
qb_red=random.sample(qb.all(), select)
# qb_red=qb.all()[1000:2000]
frames_ase=[]
for [structure_data] in qb_red:
    frame=structure_data.get_ase()
    frame.wrap(eps=1e-12)
    frames_ase.append(frame)

In [4]:
select1=300

frames_ase_random=random.sample(frames_ase, select1)

n_FPS=50
soap = SOAP(soap_type='PowerSpectrum',
           interaction_cutoff=3.5,
           max_radial=5,
           max_angular=5,
           gaussian_sigma_type='Constant',
           gaussian_sigma_constant=0.4,
           cutoff_smooth_width=0.5, 
           normalize=False)
soap_rep= soap.transform(frames_ase_random)

Xsparse = soap_rep.get_features(soap)
col_idxs = FPS(n_features_to_select=n_FPS).fit(Xsparse).selected_idx_
Xsparse = Xsparse[:, col_idxs]
Xsparse.shape, max(col_idxs)


((12398, 50), 107700)

In [ ]:
Xsparse_tot=[]
#from here, cut:

step=50
for i in range(0, len(qb_red), step):
    soap_rep= soap.transform(frames_ase[i:i+step])
    Xsp = soap_rep.get_features(soap)
    print(Xsp.shape)
    z=[ [0] * (max(col_idxs)-len(Xsp[0])+1) for _ in range(len(Xsp))]
    z=np.asarray(z)
    Xsp=np.append(Xsp, z, axis=1)
    print(Xsp.shape)
    Xsp = Xsp[:, col_idxs]
    print(Xsp.shape)
    Xsparse_tot.append(Xsp)

(2499, 34650)
(2499, 107701)
(2499, 50)
(1766, 35400)
(1766, 107701)
(1766, 50)
(2371, 40050)
(2371, 107701)
(2371, 50)
(1663, 37800)
(1663, 107701)
(1663, 50)
(1741, 35550)
(1741, 107701)
(1741, 50)
(1982, 29250)
(1982, 107701)
(1982, 50)
(2784, 35400)
(2784, 107701)
(2784, 50)
(2033, 35550)
(2033, 107701)
(2033, 50)
(2261, 32100)
(2261, 107701)
(2261, 50)
(3473, 34500)
(3473, 107701)
(3473, 50)
(2316, 33150)
(2316, 107701)
(2316, 50)
(2192, 31650)
(2192, 107701)
(2192, 50)
(1928, 35100)


In [ ]:
Xrest=np.concatenate(Xsparse_tot)

l=np.concatenate(frames_ase)

Y=np.zeros((len(l), 1))

natoms_list=[]
start=0
for frame in frames_ase:
    natoms=(len(frame))
    if natoms%4==0:
        Y[start:int(start+natoms)] = int(2)
    else:
        Y[start:int(start+natoms)] = int(1)
    start=start+natoms
print("Y shape is:"+ str(Y.shape))

In [ ]:
# Splits in train and test sets

n_train = int(len(Y)/2)  
n_test = len(Y)-n_train
r_train = np.asarray(range(len(Y)))

np.random.shuffle(r_train)

i_test = list(sorted(r_train[n_train:]))
i_train = list(sorted(r_train[:n_train]))

X_train = Xrest[i_train]
Y_train = Y[i_train]
X_test = Xrest[i_test]
Y_test = Y[i_test]

print(f'Shape of training data is: {X_train.shape}, Shape of testing data is: {X_test.shape}, ||X|| = {np.linalg.norm(X_train)}.') 
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

if X_test.shape!=X_train.shape:
    X_test=X_test[:-1,:]
    Y_test=Y_test[:-1,:]
    
if X_test.shape!=X_train.shape and X_test.shape<X_train.shape:
    X_train=X_train[:-1,:]
    Y_train=Y_train[:-1,:]  

    
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

In [ ]:
x_scaler = StandardFlexibleScaler(column_wise=False).fit(X_train)
y_scaler = StandardFlexibleScaler(column_wise=True).fit(Y_train)

# Center total dataset
X = x_scaler.transform(X)
Y = y_scaler.transform(Y)

# Center training data
X_train = x_scaler.transform(X_train)
Y_train = y_scaler.transform(Y_train)

# Center training data
X_test = x_scaler.transform(X_test)
Y_test = y_scaler.transform(Y_test)

In [ ]:
kp = KPCovR(mixing=0.8, n_components=2, **kernel_params, center=True, alpha=1E-4)
kp.fit(X_train, Y_train)
print(kp.alpha)
t = kp.transform(X_test)
y = kp.predict(X_test)
fig, ax = plt.subplots(1, 2, figsize=dbl_fig)

plot_projection(Y_test, t, title="KPCovR", fig=fig, ax=ax[0], **cmaps)
plot_regression(Y_test[:, 0], y[:, 0], title="KPCovR", fig=fig, ax=ax[1], **cmaps)

In [ ]:
K_train = gaussian_kernel(X_train, X_train)
K_test = gaussian_kernel(X_test, X_train)

k_scaler = KernelNormalizer().fit(K_train)

K_train = k_scaler.transform(K_train)
K_test = k_scaler.transform(K_test)

In [ ]:
regularization = 1e-3

krr = KernelRidge(alpha=regularization, **kernel_params)

krr.fit(K_train, Y_train)

Y_krr_train = krr.predict(K_train)
Y_krr_test = krr.predict(K_test)

table_from_dict([get_stats(x=X_train, yp=Y_krr_train, y=Y_train, k=K_train), 
                 get_stats(x=X_test,  yp=Y_krr_test, y=Y_test, k=K_test)], 
                 headers = ["Training", "Testing"], 
                 title="Kernel Ridge Regression")

In [ ]:
"""How good is the model? 1 is best"""

krr.score(K_test, Y_test)

In [ ]:
type_kernel=['linear', 'poly', 'rbf']

for i in range(len(type_kernel)):
    
    poly_kernel_svm_clf = Pipeline((
            ("scaler", StandardScaler()),
            ("svm_clf", SVC(kernel=str(type_kernel[i]), degree=5, coef0=0.9, C=5))
            ))

    print(str(type_kernel[i])+': '+ str(poly_kernel_svm_clf.fit(t, Y_test.ravel()). score(t,Y_test.ravel())))


In [ ]:
train_sizes, train_scores, valid_scores = learning_curve(
                                    SVC(kernel='linear'), Xrest, Y, train_sizes=[50, 80, 110], cv=5)